Plan for steps in this processing code
1. Define a base grid
2. Load all the tif files - know how
4. Load the shp files - know how
5. Project the shp files to tif - at the resolution / bounds, etc with the base grid - know how
6. Generate the train deposit / occurence tif files - know how
6. Unify all the tif data - know how

In [7]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import rasterio
import pandas as pd

import utilities as utils

In [2]:
tifs, shps = utils.get_input_var_files("Australia")

Loads the raster data

In [3]:
rasters = utils.load_rasters(tifs, verbosity=1)

Loading GeophysicsMagRTP_HGMDeepSources_Australia: 100%|██████████| 12/12 [00:00<00:00, 351.65it/s]

-------- GeophysicsLAB_Australia raster details --------

Raster bands and dtypes:
{1: 'float32'}


Coordinate reference system:
EPSG:4326


Bounds:BoundingBox(left=112.95, bottom=-43.65, right=153.65, top=-10.049999999999997),Size:(336, 407),Resolution:(0.1, 0.1)


-------- GeophysicsMoho_Australia raster details --------

Raster bands and dtypes:
{1: 'float32'}


Coordinate reference system:
EPSG:4326


Bounds:BoundingBox(left=112.95, bottom=-43.65, right=153.65, top=-9.349999999999994),Size:(343, 407),Resolution:(0.1, 0.1)


-------- GeophysicsSatelliteGravity_ShapeIndex_Australia raster details --------

Raster bands and dtypes:
{1: 'float32'}


Coordinate reference system:
EPSG:4326


Bounds:BoundingBox(left=113.0, bottom=-43.6, right=153.6, top=-10.600000000000001),Size:(165, 203),Resolution:(0.2, 0.2)


-------- GeophysicsGravity_Australia raster details --------

Raster bands and dtypes:
{1: 'float32'}


Coordinate reference system:
EPSG:4326


Bounds:BoundingBox(left=112.91983

Loads rasters of the vector data if available; otherwise generates them

In [4]:
try:
    rasters += utils.load_rasters(shps, verbosity=1)
except rasterio.RasterioIOError:
    base_raster = rasters[-1] # defaults to intermediate resolution raster
    vectors = utils.load_vectors(shps, verbosity=0)
    pbar = tqdm(zip(shps, vectors))
    for shp, vector in pbar:
        pbar.set_description(f"Processing {shp}")
        utils.proximity_raster_of_vector_points(base_raster, shp, vector)
    rasters += utils.load_rasters(shps, verbosity=1)

Loading DeepMagSources_Worms_Australia: 100%|██████████| 4/4 [00:00<00:00, 524.26it/s]

-------- ShallowGravitySources_Worms_Australia raster details --------

Raster bands and dtypes:
{1: 'float32'}


Coordinate reference system:
EPSG:4326


Bounds:BoundingBox(left=112.9185, bottom=-43.63974, right=153.63126, top=-10.04778000000001),Size:(4057, 4917),Resolution:(0.00828, 0.00828)


-------- DeepGravitySources_Worms_Australia raster details --------

Raster bands and dtypes:
{1: 'float32'}


Coordinate reference system:
EPSG:4326


Bounds:BoundingBox(left=112.9185, bottom=-43.63974, right=153.63126, top=-10.04778000000001),Size:(4057, 4917),Resolution:(0.00828, 0.00828)


-------- ShallowMagSources_Worms_Australia raster details --------

Raster bands and dtypes:
{1: 'float32'}


Coordinate reference system:
EPSG:4326


Bounds:BoundingBox(left=112.9185, bottom=-43.63974, right=153.63126, top=-10.04778000000001),Size:(4057, 4917),Resolution:(0.00828, 0.00828)


-------- DeepMagSources_Worms_Australia raster details --------

Raster bands and dtypes:
{1: 'float32'}


Coordi

Define a base grid for all data

In [5]:
grid_bounds = utils.compute_bounds(rasters)
print(f"Computed bounds - {grid_bounds}\n")
grid_cells = utils.region_of_cellids(grid_bounds, s2_level=12)

Computed bounds - {'bottom': -43.656585, 'top': -9.221274999999999, 'left': 112.91657, 'right': 159.100865}

Help on method from_point in module s2sphere.sphere:

from_point(point) method of builtins.type instance



Initialize the datacube

In [8]:
datacube = pd.DataFrame(data={"s2_cell_id": grid_cells})
for col in ["s2_cell_center", "s2_cell_poly"] + tifs + shps:
    datacube[col] = np.nan
datacube["mask"] = True

Populate the datacube

In [ ]:

for row_idx, row in datacube.iterrows():
    # get s2_cell_center
    # get s2_cell_polygon?
    for col_idx, col in tifs + shps:
        # sample the average (or center) values for all rasters